**The purpose of this notebook is to generate the following datasets per ontology**: 

**TRAIN**: Used for training the PFP models 
**DEV**: Used for tuning model architecture and hyperparameters beta and gamma. 
**TEST**: Use for evaluation and benchmarking 
**PLASMODIUM HOLDOUT**: Used for evaluating the model on manually curated Plamsodium functions and set FDR thresholds 


In [1]:
import numpy as np
import pandas as pd
import os 
import networkx as nx
import obonet
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO
from collections import defaultdict
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
from collections import Counter
import subprocess
import requests, sys
import json


/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#set seed for reproducibility
np.random.seed(42)

Read in the raw datasets

In [4]:
SAR_MA_AA_path = "/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/SAR_MA_AA.tsv"
Plasmodium_AA_path = "/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/Plasmodium_AA.tsv"
Plasmodium_MA_path = "/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/Plasmodium_MA.tsv"
SAR_MA_path = "/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/SAR_MA.tsv"

#read in the datasets
SAR_MA_AA = pd.read_csv(SAR_MA_AA_path, sep="\t")
Plasmodium_AA = pd.read_csv(Plasmodium_AA_path, sep="\t")
Plasmodium_MA = pd.read_csv(Plasmodium_MA_path, sep="\t")
SAR_MA = pd.read_csv(SAR_MA_path, sep="\t")
print("SAR_MA_AA:", SAR_MA_AA.shape)
print("Plasmodium_AA:", Plasmodium_AA.shape)
print("Plasmodium_MA:", Plasmodium_MA.shape)
print("SAR_MA:", SAR_MA.shape)

#assert columns are the same
assert SAR_MA_AA.columns.tolist() == Plasmodium_AA.columns.tolist() == Plasmodium_MA.columns.tolist() == SAR_MA.columns.tolist()

SAR_MA_AA: (791306, 9)
Plasmodium_AA: (102355, 9)
Plasmodium_MA: (1527, 9)
SAR_MA: (11957, 9)


In [7]:
os.chdir('./raw_data_from_uniprot')

In [5]:
def create_seq_records(data):
    seq_records = []
    for index, row in data.iterrows():
        seq_record = SeqRecord(Seq(row['Sequence']), id=row['Entry'], description=row['Organism'])
        seq_records.append(seq_record)
    assert len(seq_records) == data.shape[0]
    return seq_records

In [6]:
#first create a column in dataset1 called GOAssertion 
SAR_MA_AA["GOAssertion"] = np.nan 
Plasmodium_AA["GOAssertion"] = "AA"

SAR_MA_AA.loc[SAR_MA_AA["Entry"].isin(SAR_MA["Entry"]), "GOAssertion"] = "MA"
SAR_MA_AA.loc[SAR_MA_AA["GOAssertion"].isnull(), "GOAssertion"] = "AA"

In [7]:
total_SAR = pd.concat([SAR_MA_AA, Plasmodium_AA])
total_SAR_function = total_SAR[total_SAR["Gene Ontology (molecular function)"].notnull()]
total_SAR_process = total_SAR[total_SAR["Gene Ontology (biological process)"].notnull()]
total_SAR_component = total_SAR[total_SAR["Gene Ontology (cellular component)"].notnull()]

print(total_SAR_function.shape, total_SAR_process.shape, total_SAR_component.shape)

(716735, 10) (573293, 10) (630944, 10)


In [9]:
total_SAR_records = create_seq_records(total_SAR)
print(len(total_SAR_records))

893661


In [10]:
SeqIO.write(total_SAR_records, "total_SAR.fasta", "fasta")

893661

In [18]:
seq_records_SAR_function = create_seq_records(total_SAR_function)
seq_records_SAR_process = create_seq_records(total_SAR_process)
seq_records_SAR_component = create_seq_records(total_SAR_component)

assert len(seq_records_SAR_function) == total_SAR_function.shape[0]
assert len(seq_records_SAR_process) == total_SAR_process.shape[0]
assert len(seq_records_SAR_component) == total_SAR_component.shape[0]

In [19]:
SeqIO.write(seq_records_SAR_function, "total_SAR_function_for_reduce.fasta", "fasta")
SeqIO.write(seq_records_SAR_process, "total_SAR_process_for_reduce.fasta", "fasta")
SeqIO.write(seq_records_SAR_component, "total_SAR_component_for_reduce.fasta", "fasta")

630944

In [14]:
#run raw_data_from_uniprot/mmseq_cluster.sh

subprocess.run(["./mmseq_cluster.sh", "-h"])

Usage: ./mmseq_cluster.sh <parent_directory> <function_file> <component_file> <process_file> <min_seq_id>


CompletedProcess(args=['./mmseq_cluster.sh', '-h'], returncode=1)

In [20]:
subprocess.run(["./mmseq_cluster.sh", "reduced_90", "total_SAR_function_for_reduce.fasta", "total_SAR_component_for_reduce.fasta", "total_SAR_process_for_reduce.fasta", "0.9"])

Creating databases...
createdb /Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/total_SAR_function_for_reduce.fasta function/function_redundant_db 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[=======================================================================
Time for merging to function_redundant_db_h: 0h 0m 0s 118ms
Time for merging to function_redundant_db: 0h 0m 0s 250ms
Database type: Aminoacid
Time for processing: 0h 0m 2s 130ms
createdb /Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/total_SAR_component_for_reduce.fasta component/component_redundant_db 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting

CompletedProcess(args=['./mmseq_cluster.sh', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/total_SAR_function_for_reduce.fasta', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/total_SAR_component_for_reduce.fasta', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/total_SAR_process_for_reduce.fasta', '0.9'], returncode=0)

# Pick cluster representatives from reduced dataset (0.9)

In [22]:
SAR_function_clusters = pd.read_csv('reduced_90/function/function_clusters_redundant.tsv', sep='\t', header=None)
SAR_function_clusters.columns = ["Cluster", "Entry"]
SAR_process_clusters = pd.read_csv('reduced_90/process/process_clusters_redundant.tsv', sep='\t', header=None)
SAR_process_clusters.columns = ["Cluster", "Entry"]
SAR_component_clusters = pd.read_csv('reduced_90/component/component_clusters_redundant.tsv', sep='\t', header=None)
SAR_component_clusters.columns = ["Cluster", "Entry"]

print(SAR_function_clusters.shape)
print(SAR_process_clusters.shape)
print(SAR_component_clusters.shape)

assert SAR_function_clusters["Entry"].isin(total_SAR_function["Entry"]).all()
assert SAR_process_clusters["Entry"].isin(total_SAR_process["Entry"]).all()
assert SAR_component_clusters["Entry"].isin(total_SAR_component["Entry"]).all()

(716735, 2)
(573293, 2)
(630944, 2)


In [23]:
total_SAR_function_merged = pd.merge(total_SAR_function, SAR_function_clusters, on="Entry") 
total_SAR_process_merged = pd.merge(total_SAR_process, SAR_process_clusters, on="Entry")
total_SAR_component_merged = pd.merge(total_SAR_component, SAR_component_clusters, on="Entry")

total_SAR_function_merged_grouped = total_SAR_function_merged.groupby("Cluster")
total_SAR_process_merged_grouped = total_SAR_process_merged.groupby("Cluster")
total_SAR_component_merged_grouped = total_SAR_component_merged.groupby("Cluster")

total_SAR_function_GO = total_SAR_function_merged_grouped.apply(lambda x: x.loc[x['Gene Ontology (molecular function)'].str.findall(r"GO:\d+").str.len().idxmax()])
total_SAR_process_GO = total_SAR_process_merged_grouped.apply(lambda x: x.loc[x['Gene Ontology (biological process)'].str.findall(r"GO:\d+").str.len().idxmax()])
total_SAR_component_GO = total_SAR_component_merged_grouped.apply(lambda x: x.loc[x['Gene Ontology (cellular component)'].str.findall(r"GO:\d+").str.len().idxmax()])

print(total_SAR_function_GO.shape)
print(total_SAR_process_GO.shape)
print(total_SAR_component_GO.shape)

total_SAR_function_GO_terms = total_SAR_function_GO["Gene Ontology (molecular function)"].str.findall(r"GO:\d+")
total_SAR_process_GO_terms = total_SAR_process_GO["Gene Ontology (biological process)"].str.findall(r"GO:\d+")
total_SAR_component_GO_terms = total_SAR_component_GO["Gene Ontology (cellular component)"].str.findall(r"GO:\d+")

(352585, 11)
(279340, 11)
(306559, 11)


save 

In [24]:
#create fasta records for total_SAR_function_GO, total_SAR_process_GO, total_SAR_component_GO
seq_records_SAR_function_GO = create_seq_records(total_SAR_function_GO)
seq_records_SAR_process_GO = create_seq_records(total_SAR_process_GO)
seq_records_SAR_component_GO = create_seq_records(total_SAR_component_GO)

assert len(seq_records_SAR_function_GO) == total_SAR_function_GO.shape[0]
assert len(seq_records_SAR_process_GO) == total_SAR_process_GO.shape[0]
assert len(seq_records_SAR_component_GO) == total_SAR_component_GO.shape[0]

#write 
function_path = "reduced_90_function.fasta"
process_path = "reduced_90_process.fasta"
component_path = "reduced_90_component.fasta"

SeqIO.write(seq_records_SAR_function_GO, function_path, "fasta")
SeqIO.write(seq_records_SAR_process_GO, process_path, "fasta")
SeqIO.write(seq_records_SAR_component_GO, component_path, "fasta")

#write tsv files
total_SAR_function_GO.to_csv("reduced_90_function.tsv", sep="\t", index=False)
total_SAR_process_GO.to_csv("reduced_90_process.tsv", sep="\t", index=False)
total_SAR_component_GO.to_csv("reduced_90_component.tsv", sep="\t", index=False)

#print out the shapes into a file 
with open('reduced_90_expected_shapes.txt', 'w') as f:
    print("Function:", total_SAR_function_GO.shape, file=f)
    print("Process:", total_SAR_process_GO.shape, file=f)
    print("Component:", total_SAR_component_GO.shape, file=f)

# Propogate

In [25]:
total_SAR_function_GO_raw_terms = total_SAR_function_GO['Gene Ontology (molecular function)'].str.findall(r"GO:\d+")
total_SAR_process_GO_raw_terms = total_SAR_process_GO['Gene Ontology (biological process)'].str.findall(r"GO:\d+")
total_SAR_component_GO_raw_terms = total_SAR_component_GO['Gene Ontology (cellular component)'].str.findall(r"GO:\d+")

total_SAR_function_GO['Raw GO terms'] = total_SAR_function_GO_raw_terms
total_SAR_process_GO['Raw GO terms'] = total_SAR_process_GO_raw_terms
total_SAR_component_GO['Raw GO terms'] = total_SAR_component_GO_raw_terms

In [41]:
#get unique terms 
def get_unique_go_terms(go_terms):
    return set([term for terms in go_terms for term in terms])

unique_go_terms_function = get_unique_go_terms(total_SAR_function_GO_raw_terms)
unique_go_terms_process = get_unique_go_terms(total_SAR_process_GO_raw_terms)
unique_go_terms_component = get_unique_go_terms(total_SAR_component_GO_raw_terms)

print("Unique GO terms in function:", len(unique_go_terms_function))
print("Unique GO terms in process:", len(unique_go_terms_process))
print("Unique GO terms in component:", len(unique_go_terms_component))

#conver to lists 
unique_go_terms_function = list(unique_go_terms_function)
unique_go_terms_process = list(unique_go_terms_process)
unique_go_terms_component = list(unique_go_terms_component)

Unique GO terms in function: 2608
Unique GO terms in process: 2291
Unique GO terms in component: 810


In [57]:
base_url = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms"

# Function to get ancestors for a batch of GO terms and store the JSON response
def get_ancestors(go_terms_batch):
    terms = "%2C".join(go_terms_batch)  # Join the GO terms with URL-encoded commas
    request_url = f"{base_url}/{terms}/ancestors?relations=is_a"
    
    try:
        response = requests.get(request_url, headers={"Accept": "application/json"})
        response.raise_for_status()  # Raise exception if request failed

        data = response.json()
        
        # Build the ancestors dictionary for the batch
        ancestors_dict_batch = {}
        json_dict_batch = {}
        for result in data.get("results", []):
            term = result.get("id")
            ancestors = result.get("ancestors", [])
            ancestors_dict_batch[term] = ancestors if ancestors else None
            json_dict_batch[term] = result  # Store only the specific JSON data for each term

        return json_dict_batch, ancestors_dict_batch

    except requests.exceptions.RequestException as e:
        print(f"Error fetching ancestors for {go_terms_batch}: {e}")
        return None, None

def process_go_terms_in_batches(go_terms, batch_size=20):
    function_ancestors_dict = {}      
    function_json_dict = {}           

    # Split the GO terms into batches
    for i in tqdm(range(0, len(go_terms), batch_size)):
        batch = go_terms[i:i + batch_size]  # Get the current batch of GO terms
        json_data_batch, ancestors_dict_batch = get_ancestors(batch)

        # Update the dictionaries with the batch results
        if json_data_batch and ancestors_dict_batch:
            function_json_dict.update(json_data_batch)  # Store only the relevant JSON for each term
            function_ancestors_dict.update(ancestors_dict_batch)  # Store the ancestors

    return function_json_dict, function_ancestors_dict

batch_size = 50  

function_json_dict, function_ancestors_dict = process_go_terms_in_batches(unique_go_terms_function, batch_size=batch_size)

with open('function_ALL_data_ancestors_dict.json', 'w') as f:
    json.dump(function_ancestors_dict, f, indent=4)
with open('function_ALL_data_json_dict.json', 'w') as f:
    json.dump(function_json_dict, f, indent=4)


process_json_dict, process_ancestors_dict = process_go_terms_in_batches(unique_go_terms_process, batch_size=batch_size)
with open('process_ALL_data_ancestors_dict.json', 'w') as f:
    json.dump(process_ancestors_dict, f, indent=4)
with open('process_ALL_data_json_dict.json', 'w') as f:
    json.dump(process_json_dict, f, indent=4)

assert len(process_json_dict) == len(unique_go_terms_process)
assert len(process_ancestors_dict) == len(unique_go_terms_process)

component_json_dict, component_ancestors_dict = process_go_terms_in_batches(unique_go_terms_component, batch_size=batch_size)
with open('component_ALL_data_ancestors_dict.json', 'w') as f:
    json.dump(component_ancestors_dict, f, indent=4)
with open('component_ALL_data_json_dict.json', 'w') as f:
    json.dump(component_json_dict, f, indent=4)

assert len(component_json_dict) == len(unique_go_terms_component)
assert len(component_ancestors_dict) == len(unique_go_terms_component)

100%|██████████| 17/17 [00:13<00:00,  1.26it/s]


In [211]:
function_OBSELETE = [term for term in function_json_dict if function_json_dict[term].get("isObsolete")]
process_OBSELETE = [term for term in process_json_dict if process_json_dict[term].get("isObsolete")]
component_OBSELETE = [term for term in component_json_dict if component_json_dict[term].get("isObsolete")]

print("Function Obselete:", len(function_OBSELETE))
print("Process Obselete:", len(process_OBSELETE))
print("Component Obselete:", len(component_OBSELETE))

#combine and set 
ALL_OBSELETE_TERMS = set(function_OBSELETE + process_OBSELETE + component_OBSELETE)

print("Total Obselete terms:", len(ALL_OBSELETE_TERMS))

Function Obselete: 53
Process Obselete: 10
Component Obselete: 12
Total Obselete terms: 75


In [72]:
def add_ancestors(go_terms_list, ancestors_dict):
    all_ancestors = []
    
    for term_list in tqdm(go_terms_list, desc="Adding ancestors"):
        terms_return = set() 
        for t in term_list:
            # Check if the term exists in the dictionary and has valid ancestors
            if t in ancestors_dict and ancestors_dict[t] is not None:
                # Add ancestors, excluding the original term itself
                terms_return.update(a for a in ancestors_dict[t] if a != t)
        
        # Append the filtered ancestors found for this protein (as a list)
        all_ancestors.append(list(terms_return))

    return all_ancestors

total_SAR_function_GO_raw_terms_propogated = add_ancestors(list(total_SAR_function_GO_raw_terms), function_ancestors_dict)
total_SAR_process_GO_raw_terms_propogated = add_ancestors(list(total_SAR_process_GO_raw_terms), process_ancestors_dict)
total_SAR_component_GO_raw_terms_propogated = add_ancestors(list(total_SAR_component_GO_raw_terms), component_ancestors_dict)


Adding ancestors: 100%|██████████| 306559/306559 [00:00<00:00, 619003.57it/s]


In [76]:
#get the indicies for non-propogated terms (aka the len is 0)
non_propogated_function_terms = [i for i, x in enumerate(total_SAR_function_GO_raw_terms_propogated) if len(x) == 0]
non_propogated_process_terms = [i for i, x in enumerate(total_SAR_process_GO_raw_terms_propogated) if len(x) == 0]
non_propogated_component_terms = [i for i, x in enumerate(total_SAR_component_GO_raw_terms_propogated) if len(x) == 0]

print("Non-propogated function terms:", len(non_propogated_function_terms))
print("Non-propogated process terms:", len(non_propogated_process_terms))
print("Non-propogated component terms:", len(non_propogated_component_terms))

Non-propogated function terms: 86
Non-propogated process terms: 260
Non-propogated component terms: 1937


In [74]:
total_SAR_function_GO['Raw propagated GO terms'] = total_SAR_function_GO_raw_terms_propogated
total_SAR_process_GO['Raw propagated GO terms'] = total_SAR_process_GO_raw_terms_propogated
total_SAR_component_GO['Raw propagated GO terms'] = total_SAR_component_GO_raw_terms_propogated

In [80]:
#filter the dataframes to only include rows where the length of the Raw propagated GO terms is not zero 
total_SAR_function_GO = total_SAR_function_GO[total_SAR_function_GO['Raw propagated GO terms'].str.len() > 0]
total_SAR_process_GO = total_SAR_process_GO[total_SAR_process_GO['Raw propagated GO terms'].str.len() > 0]
total_SAR_component_GO = total_SAR_component_GO[total_SAR_component_GO['Raw propagated GO terms'].str.len() > 0]

In [82]:
seq_records_SAR_function_GO_propagated = create_seq_records(total_SAR_function_GO)
seq_records_SAR_process_GO_propagated = create_seq_records(total_SAR_process_GO)
seq_records_SAR_component_GO_propagated = create_seq_records(total_SAR_component_GO)

assert len(seq_records_SAR_function_GO_propagated) == total_SAR_function_GO.shape[0]
assert len(seq_records_SAR_process_GO_propagated) == total_SAR_process_GO.shape[0]
assert len(seq_records_SAR_component_GO_propagated) == total_SAR_component_GO.shape[0]

function_path = "reduced_90_function_propagated.fasta"
process_path = "reduced_90_process_propagated.fasta"
component_path = "reduced_90_component_propagated.fasta"

SeqIO.write(seq_records_SAR_function_GO_propagated, function_path, "fasta")
SeqIO.write(seq_records_SAR_process_GO_propagated, process_path, "fasta")
SeqIO.write(seq_records_SAR_component_GO_propagated, component_path, "fasta")

total_SAR_function_GO.to_csv("reduced_90_function_propagated.tsv", sep="\t", index=False)
total_SAR_process_GO.to_csv("reduced_90_process_propagated.tsv", sep="\t", index=False)
total_SAR_component_GO.to_csv("reduced_90_component_propagated.tsv", sep="\t", index=False)

with open('reduced_90_expected_shapes_propagated.txt', 'w') as f:
    print("Function:", total_SAR_function_GO.shape, file=f)
    print("Process:", total_SAR_process_GO.shape, file=f)
    print("Component:", total_SAR_component_GO.shape, file=f)

Now we can preform the sequence similarity splits for the train, dev, and test sets

In [83]:
#subprocess.run(["./mmseq_cluster.sh", "/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90", "/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/total_SAR_function_for_reduce.fasta", "/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/total_SAR_component_for_reduce.fasta", "/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/total_SAR_process_for_reduce.fasta", "0.9"])

subprocess.run(["./mmseq_cluster.sh", "reduced_90_similarity_30", "reduced_90_function_propagated.fasta", "reduced_90_component_propagated.fasta", "reduced_90_process_propagated.fasta", "0.3"])

Creating databases...
createdb /Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_function_propagated.fasta function/function_redundant_db 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[===================================
Time for merging to function_redundant_db_h: 0h 0m 0s 62ms
Time for merging to function_redundant_db: 0h 0m 0s 108ms
Database type: Aminoacid
Time for processing: 0h 0m 1s 237ms
createdb /Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_component_propagated.fasta component/component_redundant_db 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[=======================

CompletedProcess(args=['./mmseq_cluster.sh', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_similarity_30', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_function_propagated.fasta', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_component_propagated.fasta', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_process_propagated.fasta', '0.3'], returncode=0)

In [84]:
subprocess.run(["./mmseq_cluster.sh", "reduced_90_similarity_10", "reduced_90_function_propagated.fasta", "reduced_90_component_propagated.fasta", "reduced_90_process_propagated.fasta", "0.1"])

Creating databases...
createdb /Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_function_propagated.fasta function/function_redundant_db 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[===================================
Time for merging to function_redundant_db_h: 0h 0m 0s 64ms
Time for merging to function_redundant_db: 0h 0m 0s 138ms
Database type: Aminoacid
Time for processing: 0h 0m 1s 72ms
createdb /Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_component_propagated.fasta component/component_redundant_db 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[========================

CompletedProcess(args=['./mmseq_cluster.sh', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_similarity_10', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_function_propagated.fasta', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_component_propagated.fasta', '/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_process_propagated.fasta', '0.1'], returncode=0)

# Train, dev, test split

In [94]:
function_mmseq_out = pd.read_csv("reduced_90_similarity_30/function/function_clusters_redundant.tsv", sep='\t', header=None)
function_mmseq_out.columns = ['Cluster_2', 'Entry']

component_mmseq_out = pd.read_csv("reduced_90_similarity_30/component/component_clusters_redundant.tsv", sep='\t', header=None)
component_mmseq_out.columns = ['Cluster_2', 'Entry']

process_mmseq_out = pd.read_csv("reduced_90_similarity_30/process/process_clusters_redundant.tsv", sep='\t', header=None)
process_mmseq_out.columns = ['Cluster_2', 'Entry']

print("There are {} clusters in the function dataset.".format(function_mmseq_out['Cluster_2'].nunique()))
print("There are {} clusters in the component dataset.".format(component_mmseq_out['Cluster_2'].nunique()))
print("There are {} clusters in the process dataset.".format(process_mmseq_out['Cluster_2'].nunique()))

assert len(function_mmseq_out) == len(total_SAR_function_GO)
assert len(component_mmseq_out) == len(total_SAR_component_GO)
assert len(process_mmseq_out) == len(total_SAR_process_GO)

There are 36398 clusters in the function dataset.
There are 36571 clusters in the component dataset.
There are 30239 clusters in the process dataset.


In [95]:
#merge with the original datasets
function_merged = pd.merge(function_mmseq_out, total_SAR_function_GO, on='Entry')
component_merged = pd.merge(component_mmseq_out, total_SAR_component_GO, on='Entry')
process_merged = pd.merge(process_mmseq_out, total_SAR_process_GO, on='Entry')

assert len(function_merged) == len(function_mmseq_out)
assert len(component_merged) == len(component_mmseq_out)
assert len(process_merged) == len(process_mmseq_out)

In [87]:
#read in RNA_associated_proteins_UniProtIDs.pkl
RNA_associated_proteins = pd.read_pickle("/RNA_associated_proteins_UniProtIDs.pkl")

In [96]:
seqs_to_remove = RNA_associated_proteins

#print the original shapes 
print("Original shapes:")
print("Function:", function_merged.shape)
print("Component:", component_merged.shape)
print("Process:", process_merged.shape)

function_merged = function_merged[~function_merged['Entry'].isin(seqs_to_remove)]
component_merged = component_merged[~component_merged['Entry'].isin(seqs_to_remove)]
process_merged = process_merged[~process_merged['Entry'].isin(seqs_to_remove)]

print("After removing RNA associated proteins:")
print("Function:", function_merged.shape)
print("Component:", component_merged.shape)
print("Process:", process_merged.shape)

Original shapes:
Function: (352499, 14)
Component: (304622, 14)
Process: (279080, 14)
After removing RNA associated proteins:
Function: (352461, 14)
Component: (304567, 14)
Process: (279033, 14)


In [99]:
def prepare_data_and_split(go_terms_df, clusters_col='NA', terms_col='NA'):
    # Mapping clusters to GO terms and vice versa
    cluster_to_terms = defaultdict(list)
    term_to_clusters = defaultdict(set)

    for _, row in tqdm(go_terms_df.iterrows(), total=go_terms_df.shape[0], desc="Processing rows"):
        cluster, terms = row[clusters_col], row[terms_col]
        cluster_to_terms[cluster].extend(terms)
        for term in terms:
            term_to_clusters[term].add(cluster)

    # Identifying unique and multi-occurrence clusters
    print("Identifying unique and multi-occurrence clusters")

    unique_clusters = {cluster for cluster, terms in cluster_to_terms.items()
                       if all(len(term_to_clusters[term]) == 1 for term in terms)}
    multi_clusters = set(cluster_to_terms.keys()) - unique_clusters

    # Binarizing the GO terms
    mlb = MultiLabelBinarizer()
    all_terms = [terms for terms in cluster_to_terms.values()]
    mlb.fit(all_terms)

    # ShuffleSplit for training, validation, and test sets from multi occurrence clusters
    print("ShuffleSplit for training, validation, and test sets from multi occurrence clusters")
    ss = ShuffleSplit(n_splits=1, test_size=0.1, random_state=38)
    multi_clusters_list = list(multi_clusters)
    train_val_idx, test_idx = next(ss.split(multi_clusters_list))

    train_val_clusters = [multi_clusters_list[i] for i in train_val_idx]
    ss_train_val = ShuffleSplit(n_splits=1, test_size=0.1, random_state=38)
    train_idx, val_idx = next(ss_train_val.split(train_val_clusters))

    train_clusters = [train_val_clusters[i] for i in train_idx] + list(unique_clusters)
    val_clusters = [train_val_clusters[i] for i in val_idx]
    test_clusters = [multi_clusters_list[i] for i in test_idx]

    # Initial data extraction
    print("Extracting rows for each set")
    train_df = go_terms_df[go_terms_df[clusters_col].isin(train_clusters)]
    val_df = go_terms_df[go_terms_df[clusters_col].isin(val_clusters)]
    test_df = go_terms_df[go_terms_df[clusters_col].isin(test_clusters)]

    # Check and adjust for missing terms in training set
    train_terms = set(sum(train_df[terms_col].tolist(), []))
    val_terms = set(sum(val_df[terms_col].tolist(), []))
    test_terms = set(sum(test_df[terms_col].tolist(), []))

    missing_val_terms = val_terms - train_terms
    missing_test_terms = test_terms - train_terms

    # Move clusters with missing terms from val/test to train
    print("Moving clusters with missing terms from validation/test to training")
    if missing_val_terms:
        val_clusters_to_move = [cluster for cluster in val_clusters if any(term in missing_val_terms for term in cluster_to_terms[cluster])]
        train_clusters.extend(val_clusters_to_move)
        val_clusters = list(set(val_clusters) - set(val_clusters_to_move))

    if missing_test_terms:
        test_clusters_to_move = [cluster for cluster in test_clusters if any(term in missing_test_terms for term in cluster_to_terms[cluster])]
        train_clusters.extend(test_clusters_to_move)
        test_clusters = list(set(test_clusters) - set(test_clusters_to_move))

    # Re-extract dataframes after adjustment
    train_df = go_terms_df[go_terms_df[clusters_col].isin(train_clusters)]
    val_df = go_terms_df[go_terms_df[clusters_col].isin(val_clusters)]
    test_df = go_terms_df[go_terms_df[clusters_col].isin(test_clusters)]

    #recalculate missing val/test terms
    train_terms = set(sum(train_df[terms_col].tolist(), []))
    val_terms = set(sum(val_df[terms_col].tolist(), []))
    test_terms = set(sum(test_df[terms_col].tolist(), []))

    missing_val_terms = val_terms - train_terms
    missing_test_terms = test_terms - train_terms

    print("Validation terms not in training:", missing_val_terms)
    print("Test terms not in training:", missing_test_terms)

    ###### 
    propagated_terms_col = 'Raw propagated GO terms'  # Replace with the actual column name if different

    min_training_instances = 50 
    # Filter out terms from the training set that don't meet the minimum instance threshold
    train_terms_counter = Counter([term for terms in train_df[propagated_terms_col].tolist() for term in terms])
    terms_below_threshold = {term for term, count in train_terms_counter.items() if count < min_training_instances}

    print(f"Terms below threshold ({min_training_instances} occurrences):", terms_below_threshold)

    if terms_below_threshold:
        print(f"Filtering out {len(terms_below_threshold)} terms from the training set.")
        train_df[propagated_terms_col] = train_df[propagated_terms_col].apply(lambda terms: [term for term in terms if term not in terms_below_threshold])
        train_df = train_df[train_df[propagated_terms_col].apply(len) > 0]  # Remove rows with no terms after filtering

    # Optionally, filter validation and test sets to remove terms below the training threshold
    print("Filtering validation and test sets to remove terms below training threshold.")
    val_df[propagated_terms_col] = val_df[propagated_terms_col].apply(lambda terms: [term for term in terms if term not in terms_below_threshold])
    test_df[propagated_terms_col] = test_df[propagated_terms_col].apply(lambda terms: [term for term in terms if term not in terms_below_threshold])

    # Remove rows with no terms after filtering
    val_df = val_df[val_df[propagated_terms_col].apply(len) > 0]
    test_df = test_df[test_df[propagated_terms_col].apply(len) > 0]

    # Check the results after filtering
    filtered_train_counts = Counter([term for terms in train_df[propagated_terms_col].tolist() for term in terms])
    filtered_train_counter_terms_less_than_X = {term: count for term, count in filtered_train_counts.items() if count < min_training_instances}
    print("Filtered function training set terms with less than {} occurrences:".format(min_training_instances), filtered_train_counter_terms_less_than_X)
    print("Filtered function training set total terms:", len(filtered_train_counts))

    return train_df, val_df, test_df, mlb


function_train_df, function_val_df, function_test_df, function_mlb = prepare_data_and_split(function_merged, clusters_col='Cluster_2', terms_col='Raw GO terms')
print("Function training set shape:", function_train_df.shape)
print("Function validation set shape:", function_val_df.shape)
print("Function test set shape:", function_test_df.shape)
print("----")

Processing rows: 100%|██████████| 352461/352461 [00:09<00:00, 37125.60it/s]


Identifying unique and multi-occurrence clusters
ShuffleSplit for training, validation, and test sets from multi occurrence clusters
Extracting rows for each set
Moving clusters with missing terms from validation/test to training
Validation terms not in training: set()
Test terms not in training: set()
Terms below threshold (50 occurrences): {'GO:0170054', 'GO:0035325', 'GO:0042083', 'GO:0016723', 'GO:0015616', 'GO:0099106', 'GO:0004673', 'GO:0016815', 'GO:0015220', 'GO:0046592', 'GO:0004325', 'GO:0008920', 'GO:0034338', 'GO:0015378', 'GO:0016869', 'GO:0140359', 'GO:0002061', 'GO:0042171', 'GO:0035515', 'GO:0016664', 'GO:0019903', 'GO:0005345', 'GO:0005337', 'GO:0015296', 'GO:0043047', 'GO:0050815', 'GO:0140666', 'GO:0030291', 'GO:0033840', 'GO:0051731', 'GO:0019199', 'GO:0005343', 'GO:0036080', 'GO:0036009', 'GO:0005272', 'GO:0016885', 'GO:0015020', 'GO:0016672', 'GO:0030611', 'GO:0003905', 'GO:0033764', 'GO:0070137', 'GO:0015173', 'GO:0043394', 'GO:0030283', 'GO:0015091', 'GO:0019777

/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2548981061.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[propagated_terms_col] = train_df[propagated_terms_col].apply(lambda terms: [term for term in terms if term not in terms_below_threshold])


Filtering validation and test sets to remove terms below training threshold.


/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2548981061.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df[propagated_terms_col] = val_df[propagated_terms_col].apply(lambda terms: [term for term in terms if term not in terms_below_threshold])
/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2548981061.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[propagated_terms_col] = test_df[propagated_terms_col].apply(lambda terms: [term for term in terms if te

Filtered function training set terms with less than 50 occurrences: {}
Filtered function training set total terms: 744
Function training set shape: (289930, 14)
Function validation set shape: (29188, 14)
Function test set shape: (33343, 14)
----


In [100]:
component_train_df, component_val_df, component_test_df, component_mlb = prepare_data_and_split(component_merged, clusters_col='Cluster_2', terms_col='Raw GO terms')

Processing rows: 100%|██████████| 304567/304567 [00:08<00:00, 37979.03it/s]


Identifying unique and multi-occurrence clusters
ShuffleSplit for training, validation, and test sets from multi occurrence clusters
Extracting rows for each set
Moving clusters with missing terms from validation/test to training
Validation terms not in training: set()
Test terms not in training: set()
Terms below threshold (50 occurrences): {'GO:0005765', 'GO:0005775', 'GO:0008278', 'GO:0017053', 'GO:0033061', 'GO:0019897', 'GO:0065010', 'GO:0000775', 'GO:0005789', 'GO:0097729', 'GO:0030863', 'GO:0034705', 'GO:0000322', 'GO:0032116', 'GO:0031514', 'GO:0098802', 'GO:0038037', 'GO:0051286', 'GO:0042646', 'GO:0055028', 'GO:0005876', 'GO:0016459', 'GO:0005618', 'GO:1902687', 'GO:0098857', 'GO:0002178', 'GO:0070112', 'GO:0000164', 'GO:0097651', 'GO:0033655', 'GO:0009527', 'GO:0043190', 'GO:0030141', 'GO:0010009', 'GO:0031519', 'GO:0098636', 'GO:0097648', 'GO:0098858', 'GO:0043235', 'GO:0005657', 'GO:0034706', 'GO:0031010', 'GO:0030427', 'GO:1990391', 'GO:0036387', 'GO:0016465', 'GO:0044094

/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2548981061.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[propagated_terms_col] = train_df[propagated_terms_col].apply(lambda terms: [term for term in terms if term not in terms_below_threshold])
/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2548981061.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df[propagated_terms_col] = val_df[propagated_terms_col].apply(lambda terms: [term for term in terms if 

Filtering validation and test sets to remove terms below training threshold.
Filtered function training set terms with less than 50 occurrences: {}
Filtered function training set total terms: 255


In [101]:
process_train_df, process_val_df, process_test_df, process_mlb = prepare_data_and_split(process_merged, clusters_col='Cluster_2', terms_col='Raw GO terms')

Processing rows: 100%|██████████| 279033/279033 [00:06<00:00, 40265.36it/s]


Identifying unique and multi-occurrence clusters
ShuffleSplit for training, validation, and test sets from multi occurrence clusters
Extracting rows for each set
Moving clusters with missing terms from validation/test to training
Validation terms not in training: set()
Test terms not in training: set()
Terms below threshold (50 occurrences): {'GO:0006060', 'GO:0019835', 'GO:1901028', 'GO:0044406', 'GO:0051336', 'GO:0030335', 'GO:0043062', 'GO:0006116', 'GO:0033567', 'GO:0000103', 'GO:0009690', 'GO:1901159', 'GO:0098656', 'GO:0043447', 'GO:0015766', 'GO:0051056', 'GO:0006426', 'GO:0031589', 'GO:0017121', 'GO:0042399', 'GO:0046271', 'GO:0035725', 'GO:1902115', 'GO:0051231', 'GO:1904030', 'GO:0031554', 'GO:0070601', 'GO:0006864', 'GO:1903522', 'GO:0008016', 'GO:0009217', 'GO:0071709', 'GO:0042044', 'GO:0045595', 'GO:0010466', 'GO:0042173', 'GO:0052553', 'GO:0009303', 'GO:1900704', 'GO:0099046', 'GO:0050804', 'GO:0043954', 'GO:0033875', 'GO:0016310', 'GO:0006600', 'GO:0031365', 'GO:1901873

/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2548981061.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[propagated_terms_col] = train_df[propagated_terms_col].apply(lambda terms: [term for term in terms if term not in terms_below_threshold])
/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2548981061.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df[propagated_terms_col] = val_df[propagated_terms_col].apply(lambda terms: [term for term in terms if 

Filtering validation and test sets to remove terms below training threshold.


/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2548981061.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[propagated_terms_col] = test_df[propagated_terms_col].apply(lambda terms: [term for term in terms if term not in terms_below_threshold])


Filtered function training set terms with less than 50 occurrences: {}
Filtered function training set total terms: 1548


In [104]:
#how many GO terms are there in the propogated colmmn for function train
temp_terms = function_train_df['Raw propagated GO terms'].tolist()
#flatten the list
temp_terms = [item for sublist in temp_terms for item in sublist]
print("Total number of GO terms in the function training set:", len(temp_terms))
#unique terms
print("Total number of unique GO terms in the function training set:", len(set(temp_terms)))

Total number of GO terms in the function training set: 3096062
Total number of unique GO terms in the function training set: 744


In [64]:
#are there any terms that are not in the training set but are in the validation set and test set

temp_terms_train = function_train_df['Propagated GO terms'].tolist()
temp_terms_train = [item for sublist in temp_terms_train for item in sublist]
temp_terms_val = function_val_df['Propagated GO terms'].tolist()
temp_terms_val = [item for sublist in temp_terms_val for item in sublist]
temp_terms_test = function_test_df['Propagated GO terms'].tolist()
temp_terms_test = [item for sublist in temp_terms_test for item in sublist]

print("Terms in validation set but not in training set:", set(temp_terms_val) - set(temp_terms_train))
print("Terms in test set but not in training set:", set(temp_terms_test) - set(temp_terms_train))

Terms in validation set but not in training set: set()
Terms in test set but not in training set: set()


In [106]:
os.chdir("/PFP_PUBLISH/processed_data_90_30")
function_train_df.to_csv("function_train.tsv", sep="\t", index=False)
function_val_df.to_csv("function_val.tsv", sep="\t", index=False)
function_test_df.to_csv("function_test.tsv", sep="\t", index=False)

component_train_df.to_csv("component_train.tsv", sep="\t", index=False)
component_val_df.to_csv("component_val.tsv", sep="\t", index=False)
component_test_df.to_csv("component_test.tsv", sep="\t", index=False)

process_train_df.to_csv("process_train.tsv", sep="\t", index=False)
process_val_df.to_csv("process_val.tsv", sep="\t", index=False)
process_test_df.to_csv("process_test.tsv", sep="\t", index=False)

#write file that has contains how many unique GO terms are in the training set
function_train_terms = function_train_df['Raw propagated GO terms'].tolist()
function_train_terms = [item for sublist in function_train_terms for item in sublist]
component_train_terms = component_train_df['Raw propagated GO terms'].tolist()
component_train_terms = [item for sublist in component_train_terms for item in sublist]
process_train_terms = process_train_df['Raw propagated GO terms'].tolist()
process_train_terms = [item for sublist in process_train_terms for item in sublist]
function_train_terms_unique = len(set(function_train_terms))
component_train_terms_unique = len(set(component_train_terms))
process_train_terms_unique = len(set(process_train_terms))

with open("number_of_terms_trained_on.txt", "w") as f:
    print("Function:", len(set(function_train_terms)), file=f)
    print("Component:", len(set(component_train_terms)), file=f)
    print("Process:", len(set(process_train_terms)), file=f)

In [107]:
with open("df_shapes.txt", "w") as f:
    print("Function train:", function_train_df.shape, file=f)
    print("Function val:", function_val_df.shape, file=f)
    print("Function test:", function_test_df.shape, file=f)
    print("Component train:", component_train_df.shape, file=f)
    print("Component val:", component_val_df.shape, file=f)
    print("Component test:", component_test_df.shape, file=f)
    print("Process train:", process_train_df.shape, file=f)
    print("Process val:", process_val_df.shape, file=f)
    print("Process test:", process_test_df.shape, file=f)

In [110]:
function_merged.to_csv("presplit_function_merged.tsv", sep="\t", index=False)
component_merged.to_csv("presplit_component_merged.tsv", sep="\t", index=False)
process_merged.to_csv("presplit_process_merged.tsv", sep="\t", index=False)

In [112]:
assert len(function_train_df) + len(function_val_df) + len(function_test_df) == len(function_merged)
assert len(component_train_df) + len(component_val_df) + len(component_test_df) == len(component_merged)
assert len(process_train_df) + len(process_val_df) + len(process_test_df) == len(process_merged)

In [220]:
#create fasta files 
seq_records_function_train = create_seq_records(function_train_df)
seq_records_function_val = create_seq_records(function_val_df)
seq_records_function_test = create_seq_records(function_test_df)

seq_records_component_train = create_seq_records(component_train_df)
seq_records_component_val = create_seq_records(component_val_df)
seq_records_component_test = create_seq_records(component_test_df)

seq_records_process_train = create_seq_records(process_train_df)
seq_records_process_val = create_seq_records(process_val_df)
seq_records_process_test = create_seq_records(process_test_df)

assert len(seq_records_function_train) == function_train_df.shape[0]
assert len(seq_records_function_val) == function_val_df.shape[0]
assert len(seq_records_function_test) == function_test_df.shape[0]

assert len(seq_records_component_train) == component_train_df.shape[0]
assert len(seq_records_component_val) == component_val_df.shape[0]
assert len(seq_records_component_test) == component_test_df.shape[0]

assert len(seq_records_process_train) == process_train_df.shape[0]
assert len(seq_records_process_val) == process_val_df.shape[0]
assert len(seq_records_process_test) == process_test_df.shape[0]

In [224]:
#write out the fasta files
SeqIO.write(seq_records_function_train, "function_train.fasta", "fasta")
SeqIO.write(seq_records_function_val, "function_val.fasta", "fasta")
SeqIO.write(seq_records_function_test, "function_test.fasta", "fasta")

SeqIO.write(seq_records_component_train, "component_train.fasta", "fasta")
SeqIO.write(seq_records_component_val, "component_val.fasta", "fasta")
SeqIO.write(seq_records_component_test, "component_test.fasta", "fasta")

SeqIO.write(seq_records_process_train, "process_train.fasta", "fasta")
SeqIO.write(seq_records_process_val, "process_val.fasta", "fasta")
SeqIO.write(seq_records_process_test, "process_test.fasta", "fasta")

26746

# Recreate ancestors dict with the terms trained on 

This needs to be done since there are ancestors that are introduced that are not part of the unique GO terms. 

For example GO term X is present in the pre-propogated dataset. Y is a parent of X. Y is not a key then in the existing dict configuration. 

In [238]:
len(set(function_train_terms)), len(component_mlb.classes_), len(process_mlb.classes_)

(744, 809, 2287)

In [245]:
batch_size = 50  # You can adjust the batch size here

UNIQUE_FUNCTION_TERMS = list(set(function_train_terms))
UNIQUE_COMPONENT_TERMS = list(set(component_train_terms))
UNIQUE_PROCESS_TERMS = list(set(process_train_terms))

trained_function_json_dict, trained_function_ancestors_dict = process_go_terms_in_batches(UNIQUE_FUNCTION_TERMS, batch_size=batch_size)
assert len(trained_function_json_dict) == len(UNIQUE_FUNCTION_TERMS)

trained_component_json_dict, trained_component_ancestors_dict = process_go_terms_in_batches(UNIQUE_COMPONENT_TERMS, batch_size=batch_size)
assert len(trained_component_json_dict) == len(UNIQUE_COMPONENT_TERMS)

trained_process_json_dict, trained_process_ancestors_dict = process_go_terms_in_batches(UNIQUE_PROCESS_TERMS, batch_size=batch_size)
assert len(trained_process_json_dict) == len(UNIQUE_PROCESS_TERMS)

with open('trained_function_ALL_data_ancestors_dict.json', 'w') as f:
    json.dump(trained_function_ancestors_dict, f, indent=4)

with open('trained_function_ALL_data_json_dict.json', 'w') as f:
    json.dump(trained_function_json_dict, f, indent=4)

with open('trained_component_ALL_data_ancestors_dict.json', 'w') as f:
    json.dump(trained_component_ancestors_dict, f, indent=4)

with open('trained_component_ALL_data_json_dict.json', 'w') as f:
    json.dump(trained_component_json_dict, f, indent=4)

with open('trained_process_ALL_data_ancestors_dict.json', 'w') as f:
    json.dump(trained_process_ancestors_dict, f, indent=4)

with open('trained_process_ALL_data_json_dict.json', 'w') as f:
    json.dump(trained_process_json_dict, f, indent=4)



100%|██████████| 31/31 [00:27<00:00,  1.12it/s]


# Get embeddings

In [123]:
#function numpy array
function_all_np = np.load("TM_vec_embeddings/reduced_90_function_TM_Vec.npy", allow_pickle=True)
function_all_df = pd.read_csv("/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_function.tsv", sep='\t')
assert function_all_np.shape[0] == function_all_df.shape[0]

#component numpy array
component_all_np = np.load("TM_vec_embeddings/reduced_90_component_TM_Vec.npy", allow_pickle=True)
component_all_df = pd.read_csv("/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_component.tsv", sep='\t')
assert component_all_np.shape[0] == component_all_df.shape[0]

#process numpy array
process_all_np = np.load("TM_vec_embeddings/reduced_90_process_TM_Vec.npy", allow_pickle=True)
process_all_df = pd.read_csv("/Users/harsh/Documents/PFP_PUBLISH/raw_data_from_uniprot/reduced_90_process.tsv", sep='\t')
assert process_all_np.shape[0] == process_all_df.shape[0]

In [124]:
function_all_dict = dict(zip(function_all_df['Entry'], function_all_np))
component_all_dict = dict(zip(component_all_df['Entry'], component_all_np))
process_all_dict = dict(zip(process_all_df['Entry'], process_all_np))

function_train_np = np.array([function_all_dict[entry] for entry in function_train_df['Entry']])
function_val_np = np.array([function_all_dict[entry] for entry in function_val_df['Entry']])
function_test_np = np.array([function_all_dict[entry] for entry in function_test_df['Entry']])

component_train_np = np.array([component_all_dict[entry] for entry in component_train_df['Entry']])
component_val_np = np.array([component_all_dict[entry] for entry in component_val_df['Entry']])
component_test_np = np.array([component_all_dict[entry] for entry in component_test_df['Entry']])

process_train_np = np.array([process_all_dict[entry] for entry in process_train_df['Entry']])
process_val_np = np.array([process_all_dict[entry] for entry in process_val_df['Entry']])
process_test_np = np.array([process_all_dict[entry] for entry in process_test_df['Entry']])

assert function_train_np.shape[0] == function_train_df.shape[0]
assert function_val_np.shape[0] == function_val_df.shape[0]
assert function_test_np.shape[0] == function_test_df.shape[0]

assert component_train_np.shape[0] == component_train_df.shape[0]
assert component_val_np.shape[0] == component_val_df.shape[0]
assert component_test_np.shape[0] == component_test_df.shape[0]

assert process_train_np.shape[0] == process_train_df.shape[0]
assert process_val_np.shape[0] == process_val_df.shape[0]
assert process_test_np.shape[0] == process_test_df.shape[0]



In [126]:
np.save("function_train.npy", function_train_np)
np.save("function_val.npy", function_val_np)
np.save("function_test.npy", function_test_np)

np.save("component_train.npy", component_train_np)
np.save("component_val.npy", component_val_np)
np.save("component_test.npy", component_test_np)

np.save("process_train.npy", process_train_np)
np.save("process_val.npy", process_val_np)
np.save("process_test.npy", process_test_np)

#file with expected shapes of embeddings 
with open("expected_shapes.txt", "w") as f:
    print("Function train:", function_train_np.shape, file=f)
    print("Function val:", function_val_np.shape, file=f)
    print("Function test:", function_test_np.shape, file=f)
    print("Component train:", component_train_np.shape, file=f)
    print("Component val:", component_val_np.shape, file=f)
    print("Component test:", component_test_np.shape, file=f)
    print("Process train:", process_train_np.shape, file=f)
    print("Process val:", process_val_np.shape, file=f)
    print("Process test:", process_test_np.shape, file=f)

# Work with the Plasmodium Sequences now 

In [142]:
function_train_terms = function_train_df['Raw propagated GO terms'].tolist()
function_train_terms = [item for sublist in function_train_terms for item in sublist]
function_train_terms = list(set(function_train_terms))
print("Total number of GO terms in the function training set:", len(function_train_terms))

component_train_terms = component_train_df['Raw propagated GO terms'].tolist()
component_train_terms = [item for sublist in component_train_terms for item in sublist]
component_train_terms = list(set(component_train_terms))
print("Total number of GO terms in the component training set:", len(component_train_terms))

process_train_terms = process_train_df['Raw propagated GO terms'].tolist()
process_train_terms = [item for sublist in process_train_terms for item in sublist]
process_train_terms = list(set(process_train_terms))
print("Total number of GO terms in the process training set:", len(process_train_terms))


Total number of GO terms in the function training set: 744
Total number of GO terms in the component training set: 255
Total number of GO terms in the process training set: 1548


In [227]:
#sequences for Plasmodium MA to embedd 
Plasmodium_MA_seqs = create_seq_records(Plasmodium_MA)
SeqIO.write(Plasmodium_MA_seqs, "Plasmodium_MA_ALL.fasta", "fasta")

Plasmodium_MA_function = Plasmodium_MA[Plasmodium_MA["Gene Ontology (molecular function)"].notnull()]
Plasmodium_MA_process = Plasmodium_MA[Plasmodium_MA["Gene Ontology (biological process)"].notnull()]
Plasmodium_MA_component = Plasmodium_MA[Plasmodium_MA["Gene Ontology (cellular component)"].notnull()]

print(Plasmodium_MA_function.shape)
print(Plasmodium_MA_process.shape)
print(Plasmodium_MA_component.shape)

(1328, 9)
(1360, 9)
(1413, 9)


In [228]:
#propogate the annotations for each of the datasets
Plasmodium_MA_function['Raw GO terms'] = Plasmodium_MA_function['Gene Ontology (molecular function)'].str.findall(r"GO:\d+")
Plasmodium_MA_process['Raw GO terms'] = Plasmodium_MA_process['Gene Ontology (biological process)'].str.findall(r"GO:\d+")
Plasmodium_MA_component['Raw GO terms'] = Plasmodium_MA_component['Gene Ontology (cellular component)'].str.findall(r"GO:\d+")

Plasmodium_MA_function_raw_terms_propogated = add_ancestors(list(Plasmodium_MA_function['Raw GO terms']), function_ancestors_dict)
Plasmodium_MA_process_raw_terms_propogated = add_ancestors(list(Plasmodium_MA_process['Raw GO terms']), process_ancestors_dict)
Plasmodium_MA_component_raw_terms_propogated = add_ancestors(list(Plasmodium_MA_component['Raw GO terms']), component_ancestors_dict)

Plasmodium_MA_function['Raw propagated GO terms'] = Plasmodium_MA_function_raw_terms_propogated
Plasmodium_MA_process['Raw propagated GO terms'] = Plasmodium_MA_process_raw_terms_propogated
Plasmodium_MA_component['Raw propagated GO terms'] = Plasmodium_MA_component_raw_terms_propogated

/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2354573566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Plasmodium_MA_function['Raw GO terms'] = Plasmodium_MA_function['Gene Ontology (molecular function)'].str.findall(r"GO:\d+")
/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2354573566.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Plasmodium_MA_process['Raw GO terms'] = Plasmodium_MA_process['Gene Ontology (biological process)'].str.findall(r"GO:\d+")
/var/fol

Adding ancestors: 100%|██████████| 1413/1413 [00:00<00:00, 300382.74it/s]
/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2354573566.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Plasmodium_MA_function['Raw propagated GO terms'] = Plasmodium_MA_function_raw_terms_propogated
/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/2354573566.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Plasmodium_MA_process['Raw propagated GO terms'] = Plasmodium_MA_process_raw_terms_p

In [232]:
#In the propogated terms, how many unique terms are present there but not in the training set
Plasmodium_MA_function_raw_terms_propogated_unique = Plasmodium_MA_function['Raw propagated GO terms'].tolist()
Plasmodium_MA_function_raw_terms_propogated_unique = [item for sublist in Plasmodium_MA_function_raw_terms_propogated_unique for item in sublist]
Plasmodium_MA_function_raw_terms_propogated_unique = list(set(Plasmodium_MA_function_raw_terms_propogated_unique))
unique_terms_not_in_training_function = set(Plasmodium_MA_function_raw_terms_propogated_unique) - set(function_train_terms)
print("Unique terms in Plasmodium MA function propogated terms but not in the training set:", len(unique_terms_not_in_training_function))

Plasmodium_MA_process_raw_terms_propogated = Plasmodium_MA_process['Raw propagated GO terms'].tolist()
Plasmodium_MA_process_raw_terms_propogated = [item for sublist in Plasmodium_MA_process_raw_terms_propogated for item in sublist]
Plasmodium_MA_process_raw_terms_propogated = list(set(Plasmodium_MA_process_raw_terms_propogated))
unique_terms_not_in_training_process = set(Plasmodium_MA_process_raw_terms_propogated) - set(process_train_terms)
print("Unique terms in Plasmodium MA process propogated terms but not in the training set:", len(unique_terms_not_in_training_process))

Plasmodium_MA_component_raw_terms_propogated = Plasmodium_MA_component['Raw propagated GO terms'].tolist()
Plasmodium_MA_component_raw_terms_propogated = [item for sublist in Plasmodium_MA_component_raw_terms_propogated for item in sublist]
Plasmodium_MA_component_raw_terms_propogated = list(set(Plasmodium_MA_component_raw_terms_propogated))
unique_terms_not_in_training_component = set(Plasmodium_MA_component_raw_terms_propogated) - set(component_train_terms)
print("Unique terms in Plasmodium MA component propogated terms but not in the training set:", len(unique_terms_not_in_training_component))

Unique terms in Plasmodium MA function propogated terms but not in the training set: 55
Unique terms in Plasmodium MA process propogated terms but not in the training set: 233
Unique terms in Plasmodium MA component propogated terms but not in the training set: 21


In [154]:
# #filter out terms in raw propogated terms that are not in the unique_terms_not_in_training_function
# Plasmodium_MA_function['Raw propagated GO terms'] = Plasmodium_MA_function['Raw propagated GO terms'].apply(lambda terms: [term for term in terms if term not in unique_terms_not_in_training_function])
# Plasmodium_MA_process['Raw propagated GO terms'] = Plasmodium_MA_process['Raw propagated GO terms'].apply(lambda terms: [term for term in terms if term not in unique_terms_not_in_training_function])
# Plasmodium_MA_component['Raw propagated GO terms'] = Plasmodium_MA_component['Raw propagated GO terms'].apply(lambda terms: [term for term in terms if term not in unique_terms_not_in_training_function])

/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/997023908.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Plasmodium_MA_function['Raw propagated GO terms'] = Plasmodium_MA_function['Raw propagated GO terms'].apply(lambda terms: [term for term in terms if term not in unique_terms_not_in_training_function])
/var/folders/ky/jqpqz88565l8qdqz4jf3387c0000gn/T/ipykernel_61963/997023908.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Plasmodium_MA_process['Raw propagated GO terms'] = Plasmo

In [229]:
#first print the original shape 
print("Original shapes:")
print("Function:", Plasmodium_MA_function.shape)
print("Process:", Plasmodium_MA_process.shape)
print("Component:", Plasmodium_MA_component.shape)

Plasmodium_MA_function = Plasmodium_MA_function[Plasmodium_MA_function['Raw propagated GO terms'].str.len() > 0]
Plasmodium_MA_process = Plasmodium_MA_process[Plasmodium_MA_process['Raw propagated GO terms'].str.len() > 0]
Plasmodium_MA_component = Plasmodium_MA_component[Plasmodium_MA_component['Raw propagated GO terms'].str.len() > 0]

print("After filtering out terms not in the training set:")
print("Function:", Plasmodium_MA_function.shape)
print("Process:", Plasmodium_MA_process.shape)
print("Component:", Plasmodium_MA_component.shape)

Original shapes:
Function: (1328, 11)
Process: (1360, 11)
Component: (1413, 11)
After filtering out terms not in the training set:
Function: (1328, 11)
Process: (1355, 11)
Component: (1410, 11)


In [230]:
Plasmodium_MA_embedd_all = np.load("/Users/harsh/Documents/PFP_PUBLISH/processed_data_90_30/TM_vec_embeddings/Plasmodium_MA_TM_Vec.npy", allow_pickle=True)
print(Plasmodium_MA_embedd_all.shape)
Plasmodium_MA_ids = Plasmodium_MA['Entry'].tolist()

Plasmodium_MA_dict = dict(zip(Plasmodium_MA_ids, Plasmodium_MA_embedd_all))

(1527, 512)


In [231]:
#filter the dict based on the entries in the Plasmodium_MA_function, Plasmodium_MA_process, and Plasmodium_MA_component
Plasmodium_MA_function_np = np.array([Plasmodium_MA_dict[entry] for entry in Plasmodium_MA_function['Entry']])
Plasmodium_MA_process_np = np.array([Plasmodium_MA_dict[entry] for entry in Plasmodium_MA_process['Entry']])
Plasmodium_MA_component_np = np.array([Plasmodium_MA_dict[entry] for entry in Plasmodium_MA_component['Entry']])

assert Plasmodium_MA_function_np.shape[0] == Plasmodium_MA_function.shape[0]
assert Plasmodium_MA_process_np.shape[0] == Plasmodium_MA_process.shape[0]
assert Plasmodium_MA_component_np.shape[0] == Plasmodium_MA_component.shape[0]

np.save("Plasmodium_MA_function.npy", Plasmodium_MA_function_np)
np.save("Plasmodium_MA_process.npy", Plasmodium_MA_process_np)
np.save("Plasmodium_MA_component.npy", Plasmodium_MA_component_np)

#save the dataframes
Plasmodium_MA_function.to_csv("Plasmodium_MA_function.tsv", sep="\t", index=False)
Plasmodium_MA_process.to_csv("Plasmodium_MA_process.tsv", sep="\t", index=False)
Plasmodium_MA_component.to_csv("Plasmodium_MA_component.tsv", sep="\t", index=False)


In [233]:
Plasmodium_MA_function_seqs = create_seq_records(Plasmodium_MA_function)
Plasmodium_MA_process_seqs = create_seq_records(Plasmodium_MA_process)
Plasmodium_MA_component_seqs = create_seq_records(Plasmodium_MA_component)

In [234]:
SeqIO.write(Plasmodium_MA_function_seqs, "Plasmodium_MA_function.fasta", "fasta")
SeqIO.write(Plasmodium_MA_process_seqs, "Plasmodium_MA_process.fasta", "fasta")
SeqIO.write(Plasmodium_MA_component_seqs, "Plasmodium_MA_component.fasta", "fasta")

1410

# Generate IA TSV using the training data 

In [212]:
function_train_df_subset = total_SAR_function_GO[['Entry', 'Raw GO terms']].copy()
function_train_df_subset.columns = ['EntryID', 'term']

component_train_df_subset = total_SAR_component_GO[['Entry', 'Raw GO terms']].copy()
component_train_df_subset.columns = ['EntryID', 'term']

process_train_df_subset = total_SAR_process_GO[['Entry', 'Raw GO terms']].copy()
process_train_df_subset.columns = ['EntryID', 'term']

# Adding aspect to each respective dataframe
function_train_df_subset['aspect'] = 'MFO'
component_train_df_subset['aspect'] = 'CCO'
process_train_df_subset['aspect'] = 'BPO'

# Concatenating all dataframes to create a final combined dataframe
combined_df = pd.concat([function_train_df_subset, component_train_df_subset, process_train_df_subset], ignore_index=True)

exploded_df = combined_df.explode('term')

exploded_df.head()

#REMOVE ALL_OBSELETE_TERMS
exploded_df = exploded_df[~exploded_df['term'].isin(ALL_OBSELETE_TERMS)]

In [213]:
exploded_df.to_csv("terms_for_IA_calculation.tsv", sep="\t", index=False)


python ia.py -a /Users/harsh/Documents/PFP_PUBLISH/terms_for_IA_calculation.tsv -o /Users/harsh/Documents/PFP_PUBLISH/IA_all.tsv -g /Users/harsh/Documents/PFP_PUBLISH/go-basic.obo